In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, isnull, lit, length, explode, count, upper, lower, regexp_replace, regexp_extract

In [24]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Data quality check clientes ") \
    .config("spark.sql.parquet.enableVectorizedReader", "true") \
    .config("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED") \
    .getOrCreate()

In [25]:
df = spark.read.parquet("/app/data/raw/clientes.parquet") 

25/04/10 14:24:13 WARN SQLConf: The SQL config 'spark.sql.legacy.parquet.datetimeRebaseModeInRead' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.parquet.datetimeRebaseModeInRead' instead.
25/04/10 14:24:13 WARN SQLConf: The SQL config 'spark.sql.legacy.parquet.datetimeRebaseModeInRead' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.parquet.datetimeRebaseModeInRead' instead.
25/04/10 14:24:13 WARN SQLConf: The SQL config 'spark.sql.legacy.parquet.datetimeRebaseModeInRead' has been deprecated in Spark v3.2 and may be removed in the future. Use 'spark.sql.parquet.datetimeRebaseModeInRead' instead.


In [17]:
df.printSchema()

root
 |-- v_id_cli: string (nullable = true)
 |-- d_dt_nasc: date (nullable = true)
 |-- v_sx_cli: string (nullable = true)
 |-- n_est_cvl: string (nullable = true)



In [26]:
df.show(5)

+--------------------+----------+--------+---------+
|            v_id_cli| d_dt_nasc|v_sx_cli|n_est_cvl|
+--------------------+----------+--------+---------+
|0000009DB36F622B7639|2003-11-14|       F|     NULL|
|000000F51C15031D708E|2008-03-23|       F|     NULL|
|00000F54BE2BBF0E7B13|1986-02-24|       F|     NULL|
|000013E1FB44D9A9E50F|1984-12-13|       M|     NULL|
|00001522AD94645C7688|1992-11-10|       F|        2|
+--------------------+----------+--------+---------+
only showing top 5 rows



In [28]:
df.select([
    count(when(isnull(c), c)).alias(c) for c in df.columns
]).show()

+--------+---------+--------+---------+
|v_id_cli|d_dt_nasc|v_sx_cli|n_est_cvl|
+--------+---------+--------+---------+
|       0|     4957|   16054|    65269|
+--------+---------+--------+---------+



In [29]:
from pyspark.sql.functions import count

# Conta todas as linhas e depois as distintas
total_rows = df.count()
distinct_rows = df.distinct().count()

if total_rows > distinct_rows:
    print("Tem duplicatas!")
else:
    print("Sem duplicatas.")


Tem duplicatas!


In [19]:
spark.stop()
